In [1]:
import pandas as pd
import json
import os
import deepsig
from IPython.display import display

2024-05-08 14:12:41.588184: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 14:12:41.588230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 14:12:41.589670: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 14:12:42.785064: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [3]:
mlp_baseline_results = pd.read_csv('mlp_baseline_results.csv')
mlp_baseline_results.replace({'simple_mlp_initializer': 'Standard MLP (baseline)'}, inplace=True)

mlp_pearson_results = pd.read_csv('mlp_pearson_results.csv')
mlp_pearson_results.replace({'mlp_preg_initializer': 'Pearson regularized MLP'}, inplace=True)

mlp_spearman_results = pd.read_csv('mlp_spearman_results.csv')
mlp_spearman_results.replace({'mlp_sreg_initializer': 'Spearman regularized MLP'}, inplace=True)

mlp_auto_reg_results = pd.read_csv('mlp_auto_reg_results.csv')
mlp_auto_reg_results.replace({'mlp_auto_reg_initializer': 'Auto corr regularized MLP'}, inplace=True)

mlp_results = pd.concat([mlp_baseline_results, mlp_pearson_results, mlp_spearman_results, mlp_auto_reg_results])

ftl_baseline_results = pd.read_csv('ftl_baseline_results.csv')
ftl_baseline_results.replace({'ftl_mlp_initializer': 'Standard FTL (baseline)'}, inplace=True)

ftl_pearson_results = pd.read_csv('ftl_pearson_results.csv')
ftl_pearson_results.replace({'ftl_mlp_preg_initializer': 'Pearson regularized FTL'}, inplace=True)

ftl_spearman_results = pd.read_csv('ftl_spearman_results.csv')
ftl_spearman_results.replace({'ftl_mlp_sreg_initializer': 'Spearman regularized FTL'}, inplace=True)

ftl_results = pd.concat([ftl_baseline_results, ftl_pearson_results, ftl_spearman_results])

In [4]:
for results in [mlp_results,ftl_results]:
    results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
    results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [5]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

for results in [mlp_results,ftl_results]:
    results['Performance'] = 0
    results['Fairness'] = 0
    results['Fitness Rule'] = ''
    for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
        results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
        results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
        results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
        results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

/tmp/ipykernel_16423/3607436483.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.58517004 0.58512823 0.5840247  0.5836371  0.57625737 0.5769683
 0.58075151 0.57811729 0.57147862 0.55847176 0.56402881 0.58191431
 0.58343253 0.57740222 0.51579307 0.30958237 0.37962947 0.55899189
 0.53558872 0.2598879  0.57717035 0.50086739 0.33275221 0.37929293
 0.47729109 0.29196869 0.29037704 0.52303609 0.27480633 0.38668781
 0.50626951 0.29960979 0.4042848  0.52119218 0.2936422  0.33522388
 0.49730721 0.25112662 0.35119067 0.510052   0.26843855 0.36081008
 0.52157495 0.30114722 0.42995147 0.48267704 0.29056691 0.23139881
 0.54223416 0.23653437 0.27041017 0.52121263 0.25759348 0.23912165
 0.5223892  0.27406652 0.22303564 0.54094974 0.27699558 0.30939251
 0.51731345 0.30339828 0.37825089 0.52113734 0.26323857 0.33963196
 0.5538225  0.27850228 0.30234984 0.51877126 0.29494101 0.2648939
 0.52742877 0.24492927 0.13407928 0.53530

In [6]:
display(mlp_results)

,dataset,method,fitness_rule,fitness,Accuracy,Mathew Correlation,Equalized Odds,Statistical Parity,Equal Opportunity,solution,Performance,Fairness,Fitness Rule,Fitness Rule Abvr
0,Adult Income,Standard MLP (baseline),mcc_parity,0.392537,0.851299,0.585170,0.113127,0.192633,0.142532,{'dropout': 0.16228954240968418},0.585170,0.192633,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
1,Adult Income,Standard MLP (baseline),mcc_odds,0.473935,0.851520,0.585500,0.111565,0.198915,0.132052,{'dropout': 0.1482543145261793},0.585500,0.111565,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
2,Adult Income,Standard MLP (baseline),mcc_opportunity,0.460175,0.851410,0.585064,0.103235,0.191606,0.124889,{'dropout': 0.17861515209545933},0.585064,0.124889,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
3,Adult Income,Standard MLP (baseline),acc_parity,0.661796,0.844887,0.565373,0.082386,0.183090,0.084174,{'dropout': 0.15655371267306142},0.844887,0.183090,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
4,Adult Income,Standard MLP (baseline),acc_odds,0.766170,0.850857,0.577932,0.084687,0.180366,0.093891,{'dropout': 0.1698638434734938},0.850857,0.084687,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,German Credit,Auto corr regularized MLP,acc_odds,0.590270,0.710000,0.297044,0.119730,0.126217,0.152503,"{'l2': 0.001, 'corr_type': 'none'}",0.710000,0.119730,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)
76,German Credit,Auto corr regularized MLP,acc_opportunity,0.733810,0.760000,0.313761,0.142903,0.088294,0.026190,"{'l2': 0.0001, 'corr_type': 'spearman'}",0.760000,0.026190,Max(Accuracy - Equal Opportunity),Max(Acc - Eq. Odds)
77,Adult Income,Auto corr regularized MLP,mcc_parity,0.401731,0.850304,0.579266,0.063586,0.177534,0.051343,"{'l2': 0.01, 'corr_type': 'spearman'}",0.579266,0.177534,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
78,Adult Income,Auto corr regularized MLP,mcc_odds,0.491259,0.847430,0.569120,0.077861,0.174510,0.083271,"{'l2': 0.01, 'corr_type': 'none'}",0.569120,0.077861,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)


In [7]:
datasets = ['Adult Income', 'Bank Marketing', 'Compas Recidivism','German Credit']
datasets

['Adult Income', 'Bank Marketing', 'Compas Recidivism', 'German Credit']

In [8]:
fitness_rules = ['mcc_parity', 'mcc_opportunity', 'mcc_odds', 'acc_parity', 'acc_opportunity', 'acc_odds']
fitness_rules

['mcc_parity',
 'mcc_opportunity',
 'mcc_odds',
 'acc_parity',
 'acc_opportunity',
 'acc_odds']

In [14]:
ftl_methods = ['Standard FTL (baseline)', 'Pearson regularized FTL', 'Spearman regularized FTL']
mlp_methods = ['Standard MLP (baseline)', 'Pearson regularized MLP', 'Spearman regularized MLP', 'Auto corr regularized MLP']
significances = []
grouped_results_list = []

In [15]:
for path, methods, results in zip(['mlp_multi_aso_data_list.json', 'ftl_multi_aso_data_list.json'],
                                  [mlp_methods, ftl_methods],
                                  (mlp_results,ftl_results)):
    if os.path.exists(path):
        with open(path) as file:
            multi_aso_data_list = json.load(file)
    else:    
        multi_aso_data_list = []
        for d in datasets:
            multi_aso_data = []
            for f in fitness_rules:
                methods_results = []
                for m in methods:
                    r = results.loc[ (results['dataset'] == d) &
                                         (results['fitness_rule'] == f) &
                                         (results['method'] == m) ]\
                                .fitness.tolist()
                    if len(r) == 0:
                        r = [-1]
                    methods_results.append(r)
                min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
                multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps.tolist()})
        with open(path, 'w') as file:
            json.dump(multi_aso_data_list, file)
    
    aso_df_resume = []
    for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
        fitness_rule = aso_result['fitness_rule']
        dataset = aso_result['dataset']
    
        aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
        aso_df['method'] = methods
        aso_df['dataset'] = dataset
        aso_df['fitness_rule'] = fitness_rule
        aso_df_resume.append(aso_df)
    
    print('Significance Testing')
    significance = pd.concat(aso_df_resume)
    significance.replace(fitness_rules_abvr, inplace=True)
    significance = significance.set_index(['fitness_rule', 'dataset'])
    significance = significance.sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])
    significances.append(significance)
    
    grouped_results = results\
        .groupby(['Fitness Rule Abvr', 'dataset', 'method'])\
        .agg({'fitness': ['mean', 'std', 'count'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
        .sort_values(by=['Fitness Rule Abvr', 'dataset', ('fitness','mean')], ascending=False)
    grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'std')]:.2f})$", axis=1)
    grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'std')]:.2f})$", axis=1)
    grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'std')]:.2f})$", axis=1)
    grouped_results = grouped_results.sort_values(by=['Fitness Rule Abvr', 'dataset'])
    grouped_results_list.append(grouped_results)

Significance Testing
Significance Testing


In [11]:
significances[0].sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])

Standard MLP (baseline)  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      0.775975   
                        Adult Income                      0.549434   
                        Adult Income                      0.494774   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 1.000000   
                        German Credit                     1.000000   
                        German Credit                     0.099694   
                        German Credit                     0.877021   
                        German Credit                     0.044008   

                                           Pearson regularized MLP  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Bank Marketing                    0.939917   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 1.000000   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     0.406318   

                                           Spearman regularized MLP  \
fitness_rule            dataset                                       
Max(MCC - Stat. Parity) Adult Income                       1.000000   
                        Adult Income                       1.000000   
                        Adult Income                       1.000000   
                        Adult Income                       0.990025   
                        Bank Marketing                     0.656447   
...                                                             ...   
Max(Acc - Eq. Odds)     Compas Recidivism                  1.000000   
                        German Credit                      0.665672   
                        German Credit                      0.000000   
                        German Credit                      1.000000   
                        German Credit                      0.007765   

                                           Auto corr regularized MLP  \
fitness_rule            dataset                                        
Max(MCC - Stat. Parity) Adult Income                        1.000000   
                        Adult Income                        1.000000   
                        Adult Income                        1.000000   
                        Adult Income                        1.000000   
                        Bank Marketing                      0.589253   
...                                                              ...   
Max(Acc - Eq. Odds)     Compas Recidivism                   1.000000   
                        German Credit                       0.986402   
                        German Credit                       1.000000   
                        German Credit                       0.995571   
                        German Credit                       1.000000   

                                                              method  
fitness_rule            dataset                                       
Max(MCC - Stat. Parity) Adult Income         Standard MLP (baseline)  
                        Adult Income         Pearson regularized MLP  
                        Adult Income        Spearman regularized MLP  
     

In [16]:
grouped_results_list[0]

fitness  \
                                                                         mean   
Fitness Rule Abvr       dataset           method                                
Max(Acc - Eq. Odds)     Adult Income      Auto corr regularized MLP  0.771307   
                                          Spearman regularized MLP   0.764152   
                                          Standard MLP (baseline)    0.758290   
                                          Pearson regularized MLP    0.735901   
                        Bank Marketing    Spearman regularized MLP   0.833750   
...                                                                       ...   
Max(MCC - Stat. Parity) Compas Recidivism Standard MLP (baseline)    0.074028   
                        German Credit     Pearson regularized MLP    0.283056   
                                          Standard MLP (baseline)    0.265661   
                                          Auto corr regularized MLP  0.225252   
                                          Spearman regularized MLP   0.130942   

                                                                               \
                                                                          std   
Fitness Rule Abvr       dataset           method                                
Max(Acc - Eq. Odds)     Adult Income      Auto corr regularized MLP  0.019381   
                                          Spearman regularized MLP   0.018216   
                                          Standard MLP (baseline)    0.035161   
                                          Pearson regularized MLP    0.035771   
                        Bank Marketing    Spearman regularized MLP   0.031037   
...                                                                       ...   
Max(MCC - Stat. Parity) Compas Recidivism Standard MLP (baseline)    0.034694   
                        German Credit     Pearson regularized MLP    0.024848   
                                          Standard MLP (baseline)    0.099801   
                                          Auto corr regularized MLP  0.038090   
                                          Spearman regularized MLP   0.103555   

                                                                           \
                                                                    count   
Fitness Rule Abvr       dataset           method                            
Max(Acc - Eq. Odds)     Adult Income      Auto corr regularized MLP     3   
                                          Spearman regularized MLP      3   
                                          Standard MLP (baseline)      15   
                                          Pearson regularized MLP       3   
                        Bank Marketing    Spearman regularized MLP      3   
...                                                                   ...   
Max(MCC - Stat. Parity) Compas Recidivism Standard MLP (baseline)      30   
                        German Credit     Pearson regularized MLP       3   
                                          Standard MLP (baseline)      30   
                                          Auto corr regularized MLP     3   
                                          Spearman regularized MLP      3   

                                                                    Performance  \
                                                                           mean   
Fitness Rule Abvr       dataset           method                                  
Max(Acc - Eq. Odds)     Adult Income      Auto corr regularized MLP    0.852552   
                                          Spearman regularized MLP     0.848461   
                                          Standard MLP (baseline)      0.847864   
                                          Pearson regularized MLP      0.852294   
                        Bank Marketing    Spearman regularized MLP     0.900514   
...                                                      

In [12]:
significances[1].sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])

Standard FTL (baseline)  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      0.568838   
                        Adult Income                      1.000000   
                        Bank Marketing                    1.000000   
                        Bank Marketing                    0.347302   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 0.223670   
                        Compas Recidivism                 0.125652   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     1.000000   

                                           Pearson regularized FTL  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Bank Marketing                    1.000000   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 1.000000   
                        Compas Recidivism                 0.760271   
                        German Credit                     0.830681   
                        German Credit                     1.000000   
                        German Credit                     1.000000   

                                           Spearman regularized FTL  \
fitness_rule            dataset                                       
Max(MCC - Stat. Parity) Adult Income                       0.384582   
                        Adult Income                       0.220182   
                        Adult Income                       1.000000   
                        Bank Marketing                     1.000000   
                        Bank Marketing                     1.000000   
...                                                             ...   
Max(Acc - Eq. Odds)     Compas Recidivism                  1.000000   
                        Compas Recidivism                  1.000000   
                        German Credit                      0.000000   
                        German Credit                      0.000000   
                        German Credit                      1.000000   

                                                             method  
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income        Standard FTL (baseline)  
                        Adult Income        Pearson regularized FTL  
                        Adult Income       Spearman regularized FTL  
                        Bank Marketing      Standard FTL (baseline)  
                        Bank Marketing      Pearson regularized FTL  
...                                                             ...  
Max(Acc - Eq. Odds)     Compas Recidivism   Pearson regularized FTL  
                        Compas Recidivism  Spearman regularized FTL  
                        German Credit       Standard FTL (baseline)  
                        German Credit       Pearson regularized FTL  
                        German Credit      Spearman regularized FTL  

[72 rows x 4 columns]

In [17]:
grouped_results_list[1]

fitness  \
                                                                        mean   
Fitness Rule Abvr       dataset           method                               
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL   0.816948   
                                          Standard FTL (baseline)   0.815099   
                                          Spearman regularized FTL  0.796648   
                        Bank Marketing    Pearson regularized FTL   0.842059   
                                          Spearman regularized FTL  0.825748   
...                                                                      ...   
Max(MCC - Stat. Parity) Compas Recidivism Pearson regularized FTL   0.247669   
                                          Spearman regularized FTL  0.235185   
                        German Credit     Pearson regularized FTL   0.298430   
                                          Standard FTL (baseline)   0.256495   
                                          Spearman regularized FTL  0.183352   

                                                                              \
                                                                         std   
Fitness Rule Abvr       dataset           method                               
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL   0.023932   
                                          Standard FTL (baseline)   0.022059   
                                          Spearman regularized FTL  0.036947   
                        Bank Marketing    Pearson regularized FTL   0.037943   
                                          Spearman regularized FTL  0.051318   
...                                                                      ...   
Max(MCC - Stat. Parity) Compas Recidivism Pearson regularized FTL   0.045436   
                                          Spearman regularized FTL  0.047326   
                        German Credit     Pearson regularized FTL   0.061080   
                                          Standard FTL (baseline)   0.089117   
                                          Spearman regularized FTL  0.166189   

                                                                          \
                                                                   count   
Fitness Rule Abvr       dataset           method                           
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL     17   
                                          Standard FTL (baseline)     14   
                                          Spearman regularized FTL    13   
                        Bank Marketing    Pearson regularized FTL      5   
                                          Spearman regularized FTL     5   
...                                                                  ...   
Max(MCC - Stat. Parity) Compas Recidivism Pearson regularized FTL      5   
                                          Spearman regularized FTL     5   
                        German Credit     Pearson regularized FTL      4   
                                          Standard FTL (baseline)      4   
                                          Spearman regularized FTL     4   

                                                                   Performance  \
                                                                          mean   
Fitness Rule Abvr       dataset           method                                 
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL     0.844002   
                                          Standard FTL (baseline)     0.847485   
                                          Spearman regularized FTL    0.843398   
                        Bank Marketing    Pearson regularized FTL     0.882781   
                                          Spearman regularized FTL    0.897114   
...                                                                        ...   
Max(MCC - Stat. Parity) 